In [10]:
import sys
sys.path.append('../..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import preprocessing
from tensorflow.keras.models import save_model,load_model
from utils.preprocessing import *
from utils.gpu.cuda_cluster import *
from utils.gpu.preprocessing import read_data, factorize_small_cardinality_with_index, df_to_tfdataset, split_join, get_media_index
from utils.evaluate import calculate_ctr, compute_rce, average_precision_score
import core.config as conf

In [2]:
client

Client Scheduler: tcp://127.0.0.1:35425 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 33.47 GB


## Load data

In [27]:
# data_path = conf.raw_data_path + '*' # for all dataset
data_path = conf.raw_data_path + 'part-00000'
ori_df = read_data(data_path)

In [21]:
ori_df

,text_tokens,hashtags,tweet_id,media,links,domains,tweet_type,language,tweet_timestamp,creator_id,...,engager_id,engager_follower_count,engager_following_count,engager_is_verified,engager_account_creation,creator_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,101\t11723\t10301\t10551\t19164\t10108\t11426\...,NaN,D1E2319B8D01AA3DFCB1CA8640CBBF6E,NaN,NaN,NaN,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,1612986571,DA33978388D1B0847E5C3AC9EE647956,...,11BDF72F20D572CE1072F59D13BF6992,488,481,False,1309127975,True,NaN,NaN,NaN,NaN
1,101\t56898\t137\t152\t10320\t11403\t66618\t110...,NaN,3019F01025450ACE341309284EE6CD7B,NaN,NaN,NaN,Retweet,E7F038DE3EAD397AEC9193686C911677,1613649035,14D3668064C3ABD969097FEFEF0A79DA,...,5EF7AB46348ED999090B31ACB6DAC4A0,104,218,False,1491993346,True,NaN,NaN,NaN,NaN
2,101\t76893\t10157\t27420\t17928\t112\t10143\t4...,NaN,4BC86D2C1CF043F002AEEB4D3471039A,Photo\tPhoto,NaN,NaN,TopLevel,1F73BB863A39DB62B4A55B7E558DB1E8,1613124768,9A76E367C6FD100AE658FDB49F924018,...,89F6C949731FF6DF62BFDBF5C8144F5A,197,1488,False,1480110724,False,NaN,1.613237e+09,NaN,NaN
3,101\t56898\t137\t11135\t75225\t16994\t10129\t1...,NaN,701DF7EB9F9B88FA39CC815DBA83E708,NaN,NaN,NaN,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,1612541935,06150E2090A0BD24CA71952E4CDBD7E9,...,EB8AA255E602F40979F3BB0387B61374,99,103,False,1574979736,True,NaN,NaN,NaN,NaN
4,101\t56898\t137\t71020\t12022\t105512\t168\t13...,NaN,32C41E80C6FB39C1E9DE84FBE7A1F20B,NaN,NaN,NaN,Retweet,B8B04128918BBF54E2E178BFF1ABA833,1612572329,5C8CBD9E6C9B6C248B7F5F3535AB321A,...,C15FF2A3FF6FAE1E51691B46400C3F20,589,661,False,1380642693,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033048,101\t56898\t137\t30186\t31185\t131\t59730\t131...,NaN,6793435ABF7C38895792C5F38602D3D2,Photo,NaN,NaN,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,1612742965,D5EA09ABDF828F5C84405E7E2F3ED186,...,3A208338A26A17422851A66DE1BC4102,190,251,False,1505171126,True,NaN,NaN,NaN,1.612743e+09
3033049,101\t56898\t137\t10435\t32009\t71772\t40762\t1...,NaN,AFE949F8D1825217E730DF8D742A96E0,Photo,NaN,NaN,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,1612755638,C5062142C780A56CEB438E48F24D8E14,...,42A82659E35547E58441CF4D890F8126,541,422,False,1391538247,True,NaN,1.612756e+09,NaN,NaN
3033050,101\t100\t15582\t100\t216\t4458\t7697\t12014\t...,NaN,E0817531FB3490875A2154CCEF2349C8,Photo,NaN,NaN,TopLevel,E7F038DE3EAD397AEC9193686C911677,1613683148,13CC8D799979E6A547F633986F79F1FB,...,CF3EF27445C68D8C403A2B99B0B08419,68,93,False,1602937670,True,NaN,NaN,NaN,1.613685e+09
3033051,101\t108\t18720\t131\t14117\t35127\t11453\t130...,6028E959994E0EE130FE3C1F768B96E0\t36BC0EDEC3E6...,96CB1CE2ADD248C77E4B73FC8FC93632,Photo,NaN,NaN,TopLevel,B4DC2F82961F1263E90DF7A942CCE0B2,1612955368,6B55DBAB9BE3A0818B33A71F858C18F7,...,C023B406AF5330FBD58B48E71F52E251,125,98,False,1420182520,False,NaN,NaN,NaN,1.612969e+09


- media target encoding(photo, video, gif)
- tweet_type
- language
- tweet_timestamp
- engaged_with_user_id
- engaged_with_user_follower_count
- engaging_user_id
- engaging_user_following_count
- engagee_follows_engager
- 'reply_timestamp', 'retweet_timestamp','retweet_with_comment_timestamp', 'like_timestamp'

In [34]:
df = ori_df[['present_media', 'tweet_type', #'language', 'tweet_timestamp', 
             'engaged_with_user_follower_count', 
             'enaging_user_following_count', 'engagee_follows_engager','reply_timestamp', 'retweet_timestamp','retweet_with_comment_timestamp', 'like_timestamp']]

## Preprocessing

In [35]:
df['is_reply'] = df['reply_timestamp'].compute().applymap(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['is_retweet'] = df['retweet_timestamp'].compute().applymap(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['is_comment'] = df['retweet_with_comment_timestamp'].compute().applymap(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['is_like'] = df['like_timestamp'].compute().applymap(lambda x: 1 if x > 0 else 0).astype(np.int32)

In [36]:
df = df.drop('reply_timestamp', axis=1)
df = df.drop('retweet_timestamp', axis=1)
df = df.drop('retweet_with_comment_timestamp', axis=1)
df = df.drop('like_timestamp', axis=1)

In [37]:
df.head()

,present_media,tweet_type,engaged_with_user_follower_count,enaging_user_following_count,engagee_follows_engager,is_reply,is_retweet,is_comment,is_like
id,,,,,,,,,
1,<NA>,Retweet,2473,339,False,0,0,0,0
2,<NA>,TopLevel,4418640,1190,False,0,0,0,0
3,<NA>,TopLevel,219715,1473,False,0,0,0,1
4,<NA>,Retweet,2388283,200,False,0,0,0,0
5,Photo,TopLevel,414,379,True,0,0,0,1


### Media Target Encoding

In [38]:
df['present_media'] = df['present_media'].fillna( '' )

In [39]:
df,media_index, _ = factorize_small_cardinality_with_index(df, 'present_media', 'media_type')

In [40]:
media_index = get_media_index(media_index)

In [41]:
media_index = media_index['number_of_media'].to_frame()

In [42]:
media_index

,number_of_media
0,0
1,3
2,7
3,11
4,9
5,9
6,5
7,9
8,11
9,17


In [43]:
media_index = cudf.DataFrame(media_index)
media_index = dask_cudf.from_cudf(media_index, npartitions=2)

In [44]:
df = df.merge(media_index, left_on="media_type", right_index = True, how='left')

In [45]:
df = df.drop('present_media', axis = 1)
df = df.drop('media_type', axis = 1)

### Language Encoding

In [18]:
#df, idx_to_language, _ = factorize_small_cardinality_with_index(df, 'language', 'language_encode') # how language encoding??

In [19]:
#df = df.drop('language', axis = 1)

In [20]:
#df.head()

### Tweet Type Encoding

In [46]:
df, _= factorize_small_cardinality_with_index(df, 'tweet_type', 'tweet_type_encode') # how language encoding??

In [47]:
df = df.drop('tweet_type', axis = 1)

In [48]:
df.head()

,engaged_with_user_follower_count,enaging_user_following_count,engagee_follows_engager,is_reply,is_retweet,is_comment,is_like,number_of_media,tweet_type_encode
0,322,104,True,0,0,0,0,0,1
1,1236,357,False,0,0,0,0,0,2
2,3910,440,False,0,0,0,0,0,1
3,17009348,133,False,0,0,0,0,0,0
4,29,86,False,0,0,0,0,0,1


In [49]:
df = df.astype(np.int64)

In [50]:
df.to_parquet(conf.data_root + "preprocessing/ffnn_prep.parquet")

## Modeling

### Like

In [51]:
df_y = df[['is_reply', 'is_retweet', 'is_comment', 'is_like']]
df_x = df.drop(['is_reply', 'is_retweet', 'is_comment', 'is_like'], axis = 1)

#### Scaling

In [52]:
standard_scaler = preprocessing.StandardScaler()
standard_scaler.fit(df_x.compute().as_matrix())

StandardScaler()

In [53]:
ss = standard_scaler.transform(df_x.compute().as_matrix())

In [54]:
df_x = pd.DataFrame(ss, columns = df_x.columns)

In [55]:
X_train = df_x
y_train_like = df_y['is_like'].to_frame()

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train_like.compute(), test_size=0.2)

In [57]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2)

In [58]:
model = Sequential([
    Dense(16, activation = 'relu', input_dim = X_test.shape[1]),
    Dense(8, activation = 'relu'),
    Dense(4, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [59]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy', # softmax : sparse_categorical_crossentropy, sigmoid : binary_crossentropy
    metrics=['accuracy'])

In [60]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                96        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 273
Trainable params: 273
Non-trainable params: 0
_________________________________________________________________


In [61]:
model.fit(
    x = X_train,
    y = y_train.to_pandas(),
    validation_data=(X_val, y_val.to_pandas()),
    epochs=5,
    batch_size=64
)

Epoch 1/5
37924/37924 [==============================] - 14s 354us/step - loss: 0.6448 - accuracy: 0.6214 - val_loss: 0.6385 - val_accuracy: 0.6220
Epoch 2/5
37924/37924 [==============================] - 13s 354us/step - loss: 0.6385 - accuracy: 0.6232 - val_loss: 0.6372 - val_accuracy: 0.6244
Epoch 3/5
37924/37924 [==============================] - 13s 351us/step - loss: 0.6390 - accuracy: 0.6226 - val_loss: 0.6379 - val_accuracy: 0.6224
Epoch 4/5
37924/37924 [==============================] - 13s 351us/step - loss: 0.6383 - accuracy: 0.6237 - val_loss: 0.6369 - val_accuracy: 0.6237
Epoch 5/5
37924/37924 [==============================] - 13s 351us/step - loss: 0.6383 - accuracy: 0.6232 - val_loss: 0.6367 - val_accuracy: 0.6237


In [62]:
model.evaluate(X_test, y_test.to_pandas())

15170/15170 [==============================] - 4s 256us/step - loss: 0.6380 - accuracy: 0.6246


[0.6380455493927002, 0.6246436238288879]

## Predict

In [63]:
predict_like = model.predict(X_test)

In [64]:
predict_like

array([[0.33925655],
       [0.44314694],
       [0.3525195 ],
       ...,
       [0.36671707],
       [0.49829656],
       [0.4206782 ]], dtype=float32)

In [65]:
rce_like = compute_rce(predict_like, y_test['is_like'].to_array())
rce_like

5.104127337482045

In [66]:
average_precision_score(y_test['is_like'].to_array(), predict_like)

0.5266803760580637

In [68]:
model_path = f'/hdd/cpu_models_DNN/model-ffnn.h5'
            
save_model(model, model_path)